# 03 - Metabolic Flux Estimation

This notebook demonstrates metabolic flux estimation using FLUXestimator and COBRApy.

## Overview

Steps include:
1. Load annotated single-cell data
2. Load genome-scale metabolic model
3. Map gene expression to reactions
4. Perform flux balance analysis (FBA)
5. Analyze cell type-specific metabolism

## Setup

In [1]:
import scanpy as sc
import cobra
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
from tqdm.notebook import tqdm

# Configure
sc.settings.verbosity = 1

print(f"Scanpy version: {sc.__version__}")
print(f"COBRApy version: {cobra.__version__}")

Scanpy version: 1.11.5
COBRApy version: 0.30.0


C:\Users\dell\AppData\Local\Temp\ipykernel_5408\185862817.py:13: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('scanpy')` instead
  print(f"Scanpy version: {sc.__version__}")


## Load Configuration and Data

In [2]:
# Load configuration
with open('../config/analysis_config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Load annotated data
adata = sc.read_h5ad('../data/processed/annotated_data.h5ad')

print(f"Loaded data: {adata.shape[0]} cells x {adata.shape[1]} genes")
print(f"Cell types: {adata.obs['cell_type'].nunique()}")
adata

Loaded data: 2694 cells x 13712 genes
Cell types: 3


AnnData object with n_obs × n_vars = 2694 × 13712
    obs: 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'n_genes', 'leiden', 'cell_type'
    var: 'gene_ids', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'cell_type_colors', 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'pca', 'rank_genes', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts', 'log1p_norm'
    obsp: 'connectivities', 'distances'

## Load Metabolic Model

Load genome-scale metabolic model for mouse.

In [3]:
# Try to load a test model from BiGG database
try:
    model = cobra.io.load_model('textbook')  # Using textbook model for demo
    print(f"Loaded model: {model.id}")
    print(f"Reactions: {len(model.reactions)}")
    print(f"Metabolites: {len(model.metabolites)}")
    print(f"Genes: {len(model.genes)}")
except Exception as e:
    print(f"Could not load model: {e}")
    print("\nFor real analysis, download a mouse metabolic model:")
    print("- iMM1415 from http://bigg.ucsd.edu/models/iMM1415")
    print("- Save to data/reference/iMM1415.xml")
    print("- Load with: model = cobra.io.read_sbml_model('data/reference/iMM1415.xml')")

Loaded model: e_coli_core
Reactions: 95
Metabolites: 72
Genes: 137


In [4]:
# Inspect model structure
print("\nSample reactions:")
for rxn in list(model.reactions)[:5]:
    print(f"{rxn.id}: {rxn.reaction}")
    
print("\nObjective function:")
print(model.objective)


Sample reactions:
ACALD: acald_c + coa_c + nad_c <=> accoa_c + h_c + nadh_c
ACALDt: acald_e <=> acald_c
ACKr: ac_c + atp_c <=> actp_c + adp_c
ACONTa: cit_c <=> acon_C_c + h2o_c
ACONTb: acon_C_c + h2o_c <=> icit_c

Objective function:
Maximize
1.0*Biomass_Ecoli_core - 1.0*Biomass_Ecoli_core_reverse_2cdba


## Gene Expression Mapping

Map gene expression data to metabolic reactions.

In [5]:
def map_genes_to_reactions(model, gene_expression):
    """
    Map gene expression to reaction activity scores.
    
    Parameters:
    - model: COBRA model
    - gene_expression: pd.Series with gene names as index
    
    Returns:
    - pd.Series with reaction IDs as index and activity scores
    """
    reaction_scores = {}
    
    for reaction in model.reactions:
        if len(reaction.genes) == 0:
            # No gene association
            reaction_scores[reaction.id] = 1.0
        else:
            # Get expression of associated genes
            gene_values = []
            for gene in reaction.genes:
                gene_name = gene.name if gene.name else gene.id
                if gene_name in gene_expression.index:
                    gene_values.append(gene_expression[gene_name])
            
            if len(gene_values) > 0:
                # Use mean expression (simplified approach)
                reaction_scores[reaction.id] = np.mean(gene_values)
            else:
                reaction_scores[reaction.id] = 0.0
    
    return pd.Series(reaction_scores)

print("Gene-to-reaction mapping function defined")

Gene-to-reaction mapping function defined


## Flux Balance Analysis

Perform FBA for each cell type.

In [6]:
def perform_fba_for_cell_type(adata, cell_type, model_template, threshold=0.1):
    """
    Perform FBA for a specific cell type.
    
    Parameters:
    - adata: AnnData object
    - cell_type: Cell type label
    - model_template: COBRA model
    - threshold: Expression threshold for reaction activity
    
    Returns:
    - pd.DataFrame with flux results
    """
    # Get cells of this type
    cells_mask = adata.obs['cell_type'] == cell_type
    
    if cells_mask.sum() == 0:
        return None
    
    # Calculate mean expression
    if 'log1p_norm' in adata.layers:
        expression_data = adata[cells_mask].layers['log1p_norm']
    else:
        expression_data = adata[cells_mask].X
    
    mean_expression = np.array(expression_data.mean(axis=0)).flatten()
    gene_expression = pd.Series(mean_expression, index=adata.var_names)
    
    # Create model copy
    model = model_template.copy()
    
    # Map expression to reactions
    reaction_scores = map_genes_to_reactions(model, gene_expression)
    
    # Constrain reactions based on expression
    for reaction_id, score in reaction_scores.items():
        if score < threshold:
            reaction = model.reactions.get_by_id(reaction_id)
            reaction.lower_bound = 0
            reaction.upper_bound = 0
    
    # Perform FBA
    try:
        solution = model.optimize()
        
        if solution.status == 'optimal':
            flux_df = pd.DataFrame({
                'reaction_id': list(solution.fluxes.index),
                'flux': list(solution.fluxes.values),
                'cell_type': cell_type
            })
            return flux_df
    except Exception as e:
        print(f"FBA failed for {cell_type}: {e}")
    
    return None

print("FBA function defined")

FBA function defined


In [7]:
# Perform FBA for all cell types
cell_types = adata.obs['cell_type'].unique()
print(f"Analyzing {len(cell_types)} cell types...\n")

all_flux_results = []

for cell_type in tqdm(cell_types):
    flux_df = perform_fba_for_cell_type(adata, cell_type, model)
    if flux_df is not None:
        all_flux_results.append(flux_df)
        print(f"✓ {cell_type}: {(flux_df['flux'].abs() > 1e-6).sum()} active reactions")

# Combine results
if len(all_flux_results) > 0:
    flux_results = pd.concat(all_flux_results, ignore_index=True)
    print(f"\nTotal flux predictions: {len(flux_results)}")
else:
    print("No successful FBA runs")

Analyzing 4 cell types...



  0%|          | 0/4 [00:00<?, ?it/s]

No successful FBA runs


c:\Users\dell\anaconda3\Lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


## Analyze Results

In [8]:
if len(all_flux_results) > 0:
    # Summary statistics
    summary = flux_results.groupby('cell_type').agg({
        'flux': [
            ('mean_abs_flux', lambda x: np.abs(x).mean()),
            ('active_reactions', lambda x: (np.abs(x) > 1e-6).sum())
        ]
    })
    
    print("\nMetabolic activity summary by cell type:")
    print(summary)

In [ ]:
if len(all_flux_results) > 0:
    # Visualize flux distribution
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Active reactions per cell type
    active_rxns = flux_results.groupby('cell_type').apply(
        lambda x: (x['flux'].abs() > 1e-6).sum()
    )
    active_rxns.plot(kind='bar', ax=axes[0])
    axes[0].set_xlabel('Cell Type')
    axes[0].set_ylabel('Number of Active Reactions')
    axes[0].set_title('Metabolic Pathway Breadth')
    axes[0].tick_params(axis='x', rotation=45)
    
    # Mean absolute flux
    mean_flux = flux_results.groupby('cell_type')['flux'].apply(lambda x: np.abs(x).mean())
    mean_flux.plot(kind='bar', ax=axes[1])
    axes[1].set_xlabel('Cell Type')
    axes[1].set_ylabel('Mean Absolute Flux')
    axes[1].set_title('Average Metabolic Activity')
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()

## Save Results

In [ ]:
if len(all_flux_results) > 0:
    # Save flux results
    output_file = '../results/tables/flux_results.csv'
    flux_results.to_csv(output_file, index=False)
    
    print(f"Saved flux results to {output_file}")
    print(f"\nResults summary:")
    print(f"- Cell types analyzed: {flux_results['cell_type'].nunique()}")
    print(f"- Total flux predictions: {len(flux_results)}")
    print(f"- Reactions: {flux_results['reaction_id'].nunique()}")

## Next Steps

Proceed to notebook `04_visualization.ipynb` for comprehensive visualization of metabolic flux analysis results.